In [2]:
import networkx as nx
import numpy as np
import csv
import spacy
from networkx.algorithms.community import greedy_modularity_communities

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Graph Construction

In [29]:
# add nodes
 
G = nx.Graph()
G_copy = nx.Graph()
sentences = []
with open('N03-1017.csv','r') as csvfile:
    file = csv.reader(csvfile, delimiter="|")
    for row in file:
        sentences.append(row[1])
        #G.add_node(row[1], article_index = row[0])
        G.add_node(row[0])
        G_copy.add_node(row[0])

FileNotFoundError: [Errno 2] No such file or directory: 'N03-1017.txt'

In [4]:
'''''''''running time of this cell is about one minute'''''''''

# add edges with weight 
# rule: if the two sentences' similarity > N, these two nodes have edge

nlp = spacy.load("en_core_web_lg")
for i in range(len(sentences)-1):
    for j in range(i+1, len(sentences)):
        doc1 = nlp(sentences[i])
        doc2 = nlp(sentences[j])
        similarity = ((doc1.similarity(doc2))-0.5)*200 # 0 - 100
        if similarity > 80:
            G.add_edge(sentences[i], sentences[j], weight=similarity)
            G_copy.add_edge(sentences[i], sentences[j], weight=similarity)
print(G)
nx.write_gexf(G, "G.gexf")

Graph with 91 nodes and 1210 edges


# Current Graph Structure

In [21]:
CC_number = nx.number_connected_components(G)
print('CC_number = ' + str(CC_number))
number_of_nodes_in_CC = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
print('# nodes in connected components are ' + str(number_of_nodes_in_CC))


CC_number = 1
# nodes in connected components are [86]


In [179]:
'''degree_sequence = nx.degree_histogram(G)
print(degree_sequence)
x = np.arange(len(degree_sequence))
fig, ax = plt.subplots()
ax.scatter(x, degree_sequence)'''

'degree_sequence = nx.degree_histogram(G)\nprint(degree_sequence)\nx = np.arange(len(degree_sequence))\nfig, ax = plt.subplots()\nax.scatter(x, degree_sequence)'

In [22]:
#remove the isolated nodes
CC = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
for i in range(CC_number):
    if number_of_nodes_in_CC[i]==1:
        G.remove_nodes_from(CC[i])

print(G)
nx.write_gexf(G, "G_remove_isolated_nodes.gexf")

Graph with 86 nodes and 1210 edges


# Graph Analysis

### Clustering

In [26]:
number_of_communities = 4
c = greedy_modularity_communities(G, weight = 'weight')
all_S = []
for i in range(number_of_communities):
    all_S.append(list(c[i]))

for partition_index in range(len(all_S)):
    G.add_nodes_from(all_S[partition_index], partition = partition_index)


#nx.write_gexf(G, "G_partition.gexf")

### Extract the salient sentence in each community

##### Eigenvector centrality

In [27]:

top_sentences_in_each_community = []
for h in range(number_of_communities):
    S = nx.Graph()
    S_each = all_S[h]
    S.add_nodes_from(S_each)
    for i in range(len(S_each)-1):
        for j in range(i+1, len(S_each)):
            doc1 = nlp(S_each[i])
            doc2 = nlp(S_each[j])
            similarity = ((doc1.similarity(doc2))-0.5)*200
            if similarity>80:
                S.add_edge(S_each[i], S_each[j], weight=similarity, distance = 100-similarity)
                
    S_rank = nx.eigenvector_centrality(S, weight='weight')
    #S_rank = nx.closeness_centrality(S, distance = 'distance')
    #S_rank = nx.degree_centrality(S)
    #S_rank = nx.betweenness_centrality(S,weight='weight')
    S_rank = sorted(S_rank.items(), key=lambda item: item[1], reverse=True)
    for item in S_rank:
        G.add_node(item[0], centrality = item[1])

    
    top_sentences_in_each_community.append(S_rank[0])

#nx.write_gexf(G, "G_eigenvector.gexf")

In [28]:
# Select the most important information (centrality value < 1)

for sentence in top_sentences_in_each_community:
    if sentence[1] <1 :
        print(sentence)

(" the sentences from CNC are preprocessed by a dependency-based modification of Collins et al.'s (1999) automatic parser (with a success rate of about 80%), followed by a manual tagging procedure that is supported by a special user-friendly software tool that enables the annotators to work with (i.e. modify) the automatically derived graphic representations of the trees;", 0.22381541424078108)
(' Other related works focus on converting one grammar formalism of a treebank to another and then conducting studies on the converted treebank (Collins et al., 1999; Forst, 2003; Wang et al., 1994; Watkinson and Manandhar, 2001).', 0.2582364682759764)
(' Dependency-based representations have become increasingly popular in syntactic parsing, especially for languages that exhibit free or flexible word order, such as Czech [7], Bulgarian [15], Turkish [14] and Russian [4].', 0.5828794132587916)
(' The authors in (Collins et al., 1999) describe an approach that gives 80% accuracy in recovering unla

##### Beteenness Centrality

In [184]:
top_sentences_in_each_community = []

for h in range(number_of_communities):
    S = nx.Graph()
    S_each = all_S[h]
    S.add_nodes_from(S_each)
    for i in range(len(S_each)-1):
        for j in range(i+1, len(S_each)):
            doc1 = nlp(S_each[i])
            doc2 = nlp(S_each[j])
            similarity = ((doc1.similarity(doc2))-0.5)*200
            if similarity>80:
                S.add_edge(S_each[i], S_each[j], weight=similarity, distance = 100-similarity)
    #S_rank = nx.eigenvector_centrality(S, weight='weight')
    #S_rank = nx.closeness_centrality(S, distance = 'distance')
    #S_rank = nx.degree_centrality(S)
    S_rank = nx.betweenness_centrality(S,weight='weight')
    S_rank = sorted(S_rank.items(), key=lambda item: item[1], reverse=True)
    top_sentences_in_each_community.append(S_rank[0])

In [185]:
# Select the most important information (centrality value > 0)

for sentence in top_sentences_in_each_community:
    if sentence[1] >0 :
        print(sentence)

(" Collins et al. (1999b) proposed an algorithm to convert the Czech dependency Treebank into a phrase structure Treebank and do dependency parsing through Collins (1999a)'s model.", 0.052631578947368425)
(' It is also true of the adaptation of the Collins parser for Czech (Collins et al., 1999) and the finite-state dependency parser for Turkish by Oflazer (2003).', 0.06854838709677419)
(' Dependency-based representations have become increasingly popular in syntactic parsing, especially for languages that exhibit free or flexible word order, such as Czech [7], Bulgarian [15], Turkish [14] and Russian [4].', 0.16666666666666666)


##### Closeness Centrality

In [186]:
top_sentences_in_each_community = []

for h in range(number_of_communities):
    S = nx.Graph()
    S_each = all_S[h]
    S.add_nodes_from(S_each)
    for i in range(len(S_each)-1):
        for j in range(i+1, len(S_each)):
            doc1 = nlp(S_each[i])
            doc2 = nlp(S_each[j])
            similarity = ((doc1.similarity(doc2))-0.5)*200
            if similarity>80:
                S.add_edge(S_each[i], S_each[j], weight=similarity, distance = 100-similarity)
    #S_rank = nx.eigenvector_centrality(S, weight='weight')
    S_rank = nx.closeness_centrality(S, distance = 'distance')
    #S_rank = nx.degree_centrality(S)
    #S_rank = nx.betweenness_centrality(S,weight='weight')
    S_rank = sorted(S_rank.items(), key=lambda item: item[1], reverse=True)
    top_sentences_in_each_community.append(S_rank[0])

In [187]:
# Select the most important information (centrality value > 0)

for sentence in top_sentences_in_each_community:
    if sentence[1] >0 :
        print(sentence)

(' The remaining set of projected trees becomes the treebank that will be used to train a new dependency parser — we conduct our experiments using a version of the Collins parser that has been adapted for dependency treebanks (Collins et al. 1999).', 0.05811179519093787)
(' Interest in parsing models for languages other than English has been growing, starting with work on Czech (Collins et al., 1999) and Chinese (Bikel and Chiang, 2000; Levy and Manning, 2003).', 0.060547669912540354)
(' Dependency-based representations have become increasingly popular in syntactic parsing, especially for languages that exhibit free or flexible word order, such as Czech [7], Bulgarian [15], Turkish [14] and Russian [4].', 0.0646647384264534)
(' This is consistent with the findings of Collins et al. (1999) for Czech, where the bigram model upped dependency accuracy by about 0.9%, as well as for English where Charniak (2000) reports an increase in F-score of approximately 0.3%.', 0.07873607542856635)


##### Degree centrality

In [188]:
top_sentences_in_each_community = []

for h in range(number_of_communities):
    S = nx.Graph()
    S_each = all_S[h]
    S.add_nodes_from(S_each)
    for i in range(len(S_each)-1):
        for j in range(i+1, len(S_each)):
            doc1 = nlp(S_each[i])
            doc2 = nlp(S_each[j])
            similarity = ((doc1.similarity(doc2))-0.5)*200
            if similarity>80:
                S.add_edge(S_each[i], S_each[j], weight=similarity, distance = 100-similarity)
    #S_rank = nx.eigenvector_centrality(S, weight='weight')
    #S_rank = nx.closeness_centrality(S, distance = 'distance')
    S_rank = nx.degree_centrality(S)
    #S_rank = nx.betweenness_centrality(S,weight='weight')
    S_rank = sorted(S_rank.items(), key=lambda item: item[1], reverse=True)
    top_sentences_in_each_community.append(S_rank[0])

In [189]:
# Select the most important information (centrality value < 1)

for sentence in top_sentences_in_each_community:
    if sentence[1] <1 :
        print(sentence)

(" the sentences from CNC are preprocessed by a dependency-based modification of Collins et al.'s (1999) automatic parser (with a success rate of about 80%), followed by a manual tagging procedure that is supported by a special user-friendly software tool that enables the annotators to work with (i.e. modify) the automatically derived graphic representations of the trees;", 0.8421052631578947)
(' Other related works focus on converting one grammar formalism of a treebank to another and then conducting studies on the converted treebank (Collins et al., 1999; Forst, 2003; Wang et al., 1994; Watkinson and Manandhar, 2001).', 0.84375)
